# Adult Establishment Finder
## Introduction

When booking travel to foreign cities for a bachelor party and men are interested in the number of adult friendly establishments that are available for entertainment options.

The audience is single adult heterosexual males from 18-99

The problem is that it is not easy to gather information that would make for a great party in one place

The solution will provide a count of the number of adult-friendly establishments by city by category

## Data 

Geographic centers of major target cities to include "New York, Toronto, Montreal, Seattle, Bangkok, Tokyo, London, Singapore, Sydney" will search within 16000 meters or 10 miles of city center

Foursquare data to poll for venue categories that include "Bar, Strip Club, Massage Parlor, Night Club, Beer Bar, Casino" using the "explore" endpoint


## Methodology
Intend to use Four Square data and query venue locations at the geographic centers of each city. Then solution will extract the contents of the venue category field for "adult" themed keywords. Each venue in the adult categories will be summed and map plots will be used to show specific venues of interest. 


## Code Implementation

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import geocoder
import os
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
import IPython
from IPython.display import IFrame

In [2]:
CLIENT_ID = key=os.environ.get('FOURSQUARE_CLIENT')
CLIENT_SECRET = key=os.environ.get('FOURSQUARE_SECRET')
VERSION = '20180605' # Foursquare API version
LIMIT = 500 # A default Foursquare API limit value
RADIUS = 16000 # 10 mile radius of city center
cities = ["New York, NY","Sydney, AU","Tokyo, Japan","Montreal, Quebec","Seattle, WA","London, UK","Bangkok, Thailand","Singapore, Singapore","Las Vegas, NV"]
#Night Clubs (Bars/lounges/strip clubs, Casino, Massage Parlor, Karaoke
categories_of_interest = '4d4b7105d754a06376d81259,' + '4bf58dd8d48988d17c941735,' + '52f2ab2ebcbc57f1066b8b3c,' + '5744ccdfe4b0c0459246b4bb' 

def getNearbyVenues(city,lat, lng, radius=RADIUS):
    venues_list=[]
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT,
        categories_of_interest
    )

    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']

    # return only relevant information for each nearby venue
    venues_list.append([(
        city, 
        lat, 
        lng, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'City Latitude', 'City Longitude', 'Venue', 'Venue Latitude', 
                             'Venue Longitude', 'Venue Category']
    return(nearby_venues)

def getLatLong(city):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.bing(city,key=os.environ.get('BING_API_KEY'))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return [latitude,longitude]

best_city = cities[0]
best_city_count = 0
current_count = 0
city_stats = {}

for city in cities:
    current_count = 0
    g = getLatLong(city)
    
    city_venues = getNearbyVenues(city=city,
                                       lat=g[0],
                                       lng=g[1])
    #print(city_venues)
    
    city_stats[city] = city_venues
    city_map = folium.Map(location=g, zoom_start=11)
    
    # add markers to map
    for index, row in city_venues.iterrows():
        current_count = current_count + 1
        lat = row['Venue Latitude']
        lng = row['Venue Longitude']
        label = row['Venue'] + ', ' + row['Venue Category']  
        label = folium.Popup(label, parse_html=True)
        
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(city_map)  
    
    city_map.save(city+'.html')
    
    if current_count > best_city_count:
        best_city = city
        best_city_count = current_count

IFrame(src='New York, NY.html', width=900, height=600)
print("The best city to host the bachelor party in is ",city," because it has at least ",best_city_count," attractions to partake")

The best city to host the bachelor party in is  Las Vegas, NV  because it has at least  100  attractions to partake


### Display New York Map Markers

In [3]:
IFrame(src=cities[0]+'.html', width=900, height=600)

In [4]:
# city stats
city_st = pd.DataFrame([(0,0,0,0,0)],columns=['Bar','Casino','Nightclub','Massage','Strip Club'])

for index, row in city_stats[cities[0]].iterrows():
    if 'Casino' in row['Venue Category']: 
        city_st.loc[0,'Casino'] = city_st.loc[0,'Casino'] + 1 
    elif 'Massage' in row['Venue Category']:
        city_st.loc[0,'Massage'] = city_st.loc[0,'Massage'] + 1
    elif 'Nightclub' in row['Venue Category']:
        city_st.loc[0,'Nightclub'] = city_st.loc[0,'Nightclub'] + 1
    elif 'Strip' in row['Venue Category']:
        city_st.loc[0,'Strip Club'] = city_st.loc[0,'Strip Club'] + 1
    else: 
        city_st.loc[0,'Bar'] = city_st.loc[0,'Bar'] + 1
        
city_st.head()

,Bar,Casino,Nightclub,Massage,Strip Club
0,99,0,1,0,0


### Display Sydney Markers

In [5]:
IFrame(src=cities[1]+'.html', width=900, height=600)

In [6]:
# city stats
city_st = pd.DataFrame([(0,0,0,0,0)],columns=['Bar','Casino','Nightclub','Massage','Strip Club'])

for index, row in city_stats[cities[1]].iterrows():
    if 'Casino' in row['Venue Category']: 
        city_st.loc[0,'Casino'] = city_st.loc[0,'Casino'] + 1 
    elif 'Massage' in row['Venue Category']:
        city_st.loc[0,'Massage'] = city_st.loc[0,'Massage'] + 1
    elif 'Nightclub' in row['Venue Category']:
        city_st.loc[0,'Nightclub'] = city_st.loc[0,'Nightclub'] + 1
    elif 'Strip' in row['Venue Category']:
        city_st.loc[0,'Strip Club'] = city_st.loc[0,'Strip Club'] + 1
    else: 
        city_st.loc[0,'Bar'] = city_st.loc[0,'Bar'] + 1
        
city_st.head()

,Bar,Casino,Nightclub,Massage,Strip Club
0,100,0,0,0,0


### Display Tokyo Map Markers

In [7]:
IFrame(src=cities[2]+'.html', width=900, height=600)

In [8]:
# city stats
city_st = pd.DataFrame([(0,0,0,0,0)],columns=['Bar','Casino','Nightclub','Massage','Strip Club'])

for index, row in city_stats[cities[2]].iterrows():
    if 'Casino' in row['Venue Category']: 
        city_st.loc[0,'Casino'] = city_st.loc[0,'Casino'] + 1 
    elif 'Massage' in row['Venue Category']:
        city_st.loc[0,'Massage'] = city_st.loc[0,'Massage'] + 1
    elif 'Nightclub' in row['Venue Category']:
        city_st.loc[0,'Nightclub'] = city_st.loc[0,'Nightclub'] + 1
    elif 'Strip' in row['Venue Category']:
        city_st.loc[0,'Strip Club'] = city_st.loc[0,'Strip Club'] + 1
    else: 
        city_st.loc[0,'Bar'] = city_st.loc[0,'Bar'] + 1
        
city_st.head()

,Bar,Casino,Nightclub,Massage,Strip Club
0,94,0,6,0,0


### Display Montreal Map Markers

In [9]:
IFrame(src=cities[3]+'.html', width=900, height=600)

In [10]:
# city stats
city_st = pd.DataFrame([(0,0,0,0,0)],columns=['Bar','Casino','Nightclub','Massage','Strip Club'])

for index, row in city_stats[cities[3]].iterrows():
    if 'Casino' in row['Venue Category']: 
        city_st.loc[0,'Casino'] = city_st.loc[0,'Casino'] + 1 
    elif 'Massage' in row['Venue Category']:
        city_st.loc[0,'Massage'] = city_st.loc[0,'Massage'] + 1
    elif 'Nightclub' in row['Venue Category']:
        city_st.loc[0,'Nightclub'] = city_st.loc[0,'Nightclub'] + 1
    elif 'Strip' in row['Venue Category']:
        city_st.loc[0,'Strip Club'] = city_st.loc[0,'Strip Club'] + 1
    else: 
        city_st.loc[0,'Bar'] = city_st.loc[0,'Bar'] + 1
        
city_st.head()

,Bar,Casino,Nightclub,Massage,Strip Club
0,97,0,3,0,0


### Display Seattle Map Markers

In [11]:
IFrame(src=cities[4]+'.html', width=900, height=600)

In [12]:
# city stats
city_st = pd.DataFrame([(0,0,0,0,0)],columns=['Bar','Casino','Nightclub','Massage','Strip Club'])

for index, row in city_stats[cities[4]].iterrows():
    if 'Casino' in row['Venue Category']: 
        city_st.loc[0,'Casino'] = city_st.loc[0,'Casino'] + 1 
    elif 'Massage' in row['Venue Category']:
        city_st.loc[0,'Massage'] = city_st.loc[0,'Massage'] + 1
    elif 'Nightclub' in row['Venue Category']:
        city_st.loc[0,'Nightclub'] = city_st.loc[0,'Nightclub'] + 1
    elif 'Strip' in row['Venue Category']:
        city_st.loc[0,'Strip Club'] = city_st.loc[0,'Strip Club'] + 1
    else: 
        city_st.loc[0,'Bar'] = city_st.loc[0,'Bar'] + 1
        
city_st.head()

,Bar,Casino,Nightclub,Massage,Strip Club
0,99,0,1,0,0


### Display London Map Markers

In [13]:
IFrame(src=cities[5]+'.html', width=900, height=600)

In [14]:
# city stats
city_st = pd.DataFrame([(0,0,0,0,0)],columns=['Bar','Casino','Nightclub','Massage','Strip Club'])

for index, row in city_stats[cities[5]].iterrows():
    if 'Casino' in row['Venue Category']: 
        city_st.loc[0,'Casino'] = city_st.loc[0,'Casino'] + 1 
    elif 'Massage' in row['Venue Category']:
        city_st.loc[0,'Massage'] = city_st.loc[0,'Massage'] + 1
    elif 'Nightclub' in row['Venue Category']:
        city_st.loc[0,'Nightclub'] = city_st.loc[0,'Nightclub'] + 1
    elif 'Strip' in row['Venue Category']:
        city_st.loc[0,'Strip Club'] = city_st.loc[0,'Strip Club'] + 1
    else: 
        city_st.loc[0,'Bar'] = city_st.loc[0,'Bar'] + 1
        
city_st.head()

,Bar,Casino,Nightclub,Massage,Strip Club
0,99,0,1,0,0


### Display Bangkok Map Markers

In [15]:
IFrame(src=cities[6]+'.html', width=900, height=600)

In [16]:
# city stats
city_st = pd.DataFrame([(0,0,0,0,0)],columns=['Bar','Casino','Nightclub','Massage','Strip Club'])

for index, row in city_stats[cities[6]].iterrows():
    if 'Casino' in row['Venue Category']: 
        city_st.loc[0,'Casino'] = city_st.loc[0,'Casino'] + 1 
    elif 'Massage' in row['Venue Category']:
        city_st.loc[0,'Massage'] = city_st.loc[0,'Massage'] + 1
    elif 'Nightclub' in row['Venue Category']:
        city_st.loc[0,'Nightclub'] = city_st.loc[0,'Nightclub'] + 1
    elif 'Strip' in row['Venue Category']:
        city_st.loc[0,'Strip Club'] = city_st.loc[0,'Strip Club'] + 1
    else: 
        city_st.loc[0,'Bar'] = city_st.loc[0,'Bar'] + 1
        
city_st.head()

,Bar,Casino,Nightclub,Massage,Strip Club
0,98,0,1,0,1


### Display Singapore Map Markers

In [17]:
IFrame(src=cities[7]+'.html', width=900, height=600)

In [18]:
# city stats
city_st = pd.DataFrame([(0,0,0,0,0)],columns=['Bar','Casino','Nightclub','Massage','Strip Club'])

for index, row in city_stats[cities[7]].iterrows():
    if 'Casino' in row['Venue Category']: 
        city_st.loc[0,'Casino'] = city_st.loc[0,'Casino'] + 1 
    elif 'Massage' in row['Venue Category']:
        city_st.loc[0,'Massage'] = city_st.loc[0,'Massage'] + 1
    elif 'Nightclub' in row['Venue Category']:
        city_st.loc[0,'Nightclub'] = city_st.loc[0,'Nightclub'] + 1
    elif 'Strip' in row['Venue Category']:
        city_st.loc[0,'Strip Club'] = city_st.loc[0,'Strip Club'] + 1
    else: 
        city_st.loc[0,'Bar'] = city_st.loc[0,'Bar'] + 1
        
city_st.head()

,Bar,Casino,Nightclub,Massage,Strip Club
0,98,0,2,0,0


### Display Las Vegas Map Markers

In [19]:
IFrame(src=cities[8]+'.html', width=900, height=600)

In [20]:
# city stats
city_st = pd.DataFrame([(0,0,0,0,0)],columns=['Bar','Casino','Nightclub','Massage','Strip Club'])

for index, row in city_stats[cities[8]].iterrows():
    if 'Casino' in row['Venue Category']: 
        city_st.loc[0,'Casino'] = city_st.loc[0,'Casino'] + 1 
    elif 'Massage' in row['Venue Category']:
        city_st.loc[0,'Massage'] = city_st.loc[0,'Massage'] + 1
    elif 'Nightclub' in row['Venue Category']:
        city_st.loc[0,'Nightclub'] = city_st.loc[0,'Nightclub'] + 1
    elif 'Strip' in row['Venue Category']:
        city_st.loc[0,'Strip Club'] = city_st.loc[0,'Strip Club'] + 1
    else: 
        city_st.loc[0,'Bar'] = city_st.loc[0,'Bar'] + 1
        
city_st.head()

,Bar,Casino,Nightclub,Massage,Strip Club
0,92,0,8,0,0


## Model Findings

Unfortunately it seems like "Bars" dominate all results when 100, API limitation, results are returned. If I were to run the data pulls again, I would remove "Bars/restaurants" from the category searches. Honestly every city has these in abundance and is not interesting for adult-themed searches. Even "Coffee shops" are returned by the API. I would also pull from several lat longs in a city grid search to work on the 100 data point limitation from a single geolocation.  

## Conclusions

Glad to be finished with the Data Science track series. In this module, we reviewed API usage from a service provider and a basic unsupervised machine learning technique of Kmeans clustering.
